In [6]:
import numpy as np
import pandas as pd
from PIL import Image as im
import matplotlib.pyplot as plt
import cv2

In [7]:
# DATA_PATH = r'C:\Users\yiju\Desktop\Copy\Data\hubmap-kidney-segmentation/'
DATA_PATH = r'C:\Users\yiju\Desktop\Copy\Data\Colon_data_reprocessed/'
# Read rle prediction table (submission.csv) as df
index = 5
df = pd.read_csv(rf'C:\Users\yiju\Desktop\Copy\Scripts\1. Tom\final_result\{index}\submission_colon.csv')
# Read ground truth rles as rles
rles = pd.read_csv(DATA_PATH + 'test.csv')
df_info = pd.read_csv(DATA_PATH + 'HuBMAP-20-dataset_information.csv')

In [8]:
def enc2mask(encs, shape):
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for m, enc in enumerate(encs):
        if isinstance(enc, np.float) and np.isnan(enc):
            continue
        enc_split = enc.split()
        for i in range(len(enc_split) // 2):
            start = int(enc_split[2 * i]) - 1
            length = int(enc_split[2 * i + 1])
            img[start: start + length] = 1 + m

    return img.reshape(shape).T

In [9]:
threshold = 0.5

In [10]:
for img in rles['id'].unique():
    shape = df_info[df_info.image_file == img][['width_pixels', 'height_pixels']].values.astype(int)[0]
    
    truth = rles[rles['id'] == img]['predicted']
    mask_truth = enc2mask(truth, shape)
    
    gt_mask = cv2.resize((mask_truth).astype('uint8'), (shape[0], shape[1]))
    gt_mask = ((gt_mask>threshold)*255).astype(np.uint8)
    # fig, ax = plt.subplots(figsize=(15,15))
    # ax.imshow(gt_mask)
    # plt.savefig("gt_"+img+".png")
    cv2.imwrite("gt_"+img+".png",gt_mask)
    print("saved ", "gt_"+img+".png")
    
    pred = df[df['id'] == img]['predicted']
    mask_pred = enc2mask(pred, shape) 
    
    pred_mask = cv2.resize((mask_pred).astype('uint8'), (shape[0], shape[1]))
    pred_mask = ((pred_mask>threshold)*255).astype(np.uint8)
    # fig, ax = plt.subplots(figsize=(15,15))
    # ax.imshow(pred_mask)
    # plt.savefig("pred_"+img+".png")
    cv2.imwrite(f"pred_{index}_"+img+".png",pred_mask)
    print("saved ", f"pred_{index}_"+img+".png")

C:\Users\yiju\Anaconda3\envs\tf-gpu-37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


saved  gt_CL_HandE_1234_B004_bottomleft.png
saved  pred_5_CL_HandE_1234_B004_bottomleft.png
saved  gt_HandE_B005_CL_b_RGB_bottomleft.png
saved  pred_5_HandE_B005_CL_b_RGB_bottomleft.png
